In [1]:
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
print(np.__version__)

2.3.0
1.18.5


In [2]:
# divide uma série temporal univariada em amostras
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # encontra o final da amostra
        end_ix = i + n_steps
        # verifica se passou do final
        if end_ix > len(sequence)-1:
            break
        # junta as partes de entrada e saída do padrão
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
df = pd.read_csv('sp_beaches_update.csv', parse_dates=['Date'])
df = df.sort_values(by=['Date'])
df=df.loc[~df['Enterococcus'].isnull()]
#remover a praia do Leste, da cidade de iguape, pois esta praia sumiu por erosão em 2012
#remover a praia MASSAGUAÇU (AV. M. H. CARVALHO), da cidade de CARAGUATATUBA, não possui mais medições em 2021
#remover a praia RIO PEREQUÊ, da cidade de CUBATÃO, até dez de 2020 possuía medições semanais, depois de janeiro de 2021 as medições passaram a ser mensais
#remover as praias da cidade ILHA ANCHIETA pois não exitem mais medições em 2021
df = df.loc[df['Beach']!='DO LESTE'].loc[df['Beach']!='MASSAGUAÇU (AV. M. H. CARVALHO)'].loc[df['Beach']!='RIO PEREQUÊ'].loc[df['City']!='ILHA ANCHIETA']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71443 entries, 0 to 70616
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   City          71443 non-null  object        
 1   Beach         71443 non-null  object        
 2   Date          71443 non-null  datetime64[ns]
 3   Enterococcus  71443 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.7+ MB


In [4]:
df.loc[df['City']=='UBATUBA'].loc[df['Beach']=='GRANDE']

,City,Beach,Date,Enterococcus
7614,UBATUBA,GRANDE,2012-01-03,5
7615,UBATUBA,GRANDE,2012-01-08,42
7616,UBATUBA,GRANDE,2012-01-15,92
7617,UBATUBA,GRANDE,2012-01-22,16
7618,UBATUBA,GRANDE,2012-01-29,9
...,...,...,...,...
74360,UBATUBA,GRANDE,2021-07-19,38
74361,UBATUBA,GRANDE,2021-07-26,17
74362,UBATUBA,GRANDE,2021-08-02,1
74363,UBATUBA,GRANDE,2021-08-09,1


In [5]:
# selecionar praias com medições semanal
df_freq = pd.read_csv('frequencia_praias.csv')
df_freq = df_freq.loc[df_freq['Beach']!='DO LESTE'].loc[df_freq['Beach']!='MASSAGUAÇU (AV. M. H. CARVALHO)'].loc[df_freq['Beach']!='RIO PEREQUÊ'].loc[df_freq['City']!='ILHA ANCHIETA']
df_freq = df_freq.loc[df_freq['Frequency']=='SEMANAL']
df_freq

,City,Beach,Frequency
11,BERTIOGA,BORACÉIA - COL. MARISTA,SEMANAL
12,BERTIOGA,BORACÉIA - SUL,SEMANAL
13,BERTIOGA,GUARATUBA,SEMANAL
14,BERTIOGA,SÃO LOURENÇO (JUNTO AO MORRO),SEMANAL
15,BERTIOGA,SÃO LOURENÇO (RUA 2),SEMANAL
...,...,...,...
161,UBATUBA,DURA,SEMANAL
162,UBATUBA,LAGOINHA (R. ENGENHO VELHO),SEMANAL
163,UBATUBA,LAGOINHA (CAMPING),SEMANAL
164,UBATUBA,SAPÉ,SEMANAL


In [6]:
n_steps = 16 #16 medições representa 4 meses que serão utilizados para prever a próxima medição
#Para cada cidade 
for cidade in df_freq.City.unique():
    # Para cada praia desta cidade
    for praia in df_freq.loc[df_freq['City']==cidade].Beach.unique():
        #Imprime que iniicou o processo de carregamento de dados de uma determinada praia
        print(f'iniciando: {cidade} - {praia}')
        #Coleta os dados desta praia
        df_beach = df.loc[df['City']==cidade].loc[df['Beach']==praia][['Date','Enterococcus']]
        #formata os dados desta praia para enviar ao modelo
        df_beach.columns = ['ds', 'y']
        df_beach.set_index('ds', inplace=True)
        #divide os dados da praia em lotes de X com n_steps medições para y com 1 medição de rótulo
        X, y = split_sequence(df_beach.values, n_steps)
        #formata de acordo com a entrada do modelo
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        #prepara as camadas do modelo
        model = Sequential()
        model.add(LSTM(104, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(104, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        #imprime que o modelo irá começar a ser treinado para esta praia
        print('Treinando modelo: '+cidade+'-'+praia)
        #começa o treino do modelo na GPU
        with tf.device('/GPU:0'): 
            R = model.fit(X, y, epochs=200, verbose=0)
        #depois de treinado pega o erro do MSE
        loss=round(R.history['loss'][-1])
        #salva o modelo com o nome de modelcidade-praia.h5
        model.save('model'+cidade+'-'+praia+'.h5')
        #imprime que o modelo foi treinado e o valor do erro MSE
        print('Modelo Treinado: '+cidade+'-'+praia+'- MSE:'+str(loss))

iniciando: BERTIOGA - BORACÉIA - COL. MARISTA
Treinando modelo: BERTIOGA-BORACÉIA - COL. MARISTA
Modelo Treinado: BERTIOGA-BORACÉIA - COL. MARISTA- MSE:1781
iniciando: BERTIOGA - BORACÉIA - SUL
Treinando modelo: BERTIOGA-BORACÉIA - SUL
Modelo Treinado: BERTIOGA-BORACÉIA - SUL- MSE:1538
iniciando: BERTIOGA - GUARATUBA
Treinando modelo: BERTIOGA-GUARATUBA
Modelo Treinado: BERTIOGA-GUARATUBA- MSE:146
iniciando: BERTIOGA - SÃO LOURENÇO (JUNTO AO MORRO)
Treinando modelo: BERTIOGA-SÃO LOURENÇO (JUNTO AO MORRO)
Modelo Treinado: BERTIOGA-SÃO LOURENÇO (JUNTO AO MORRO)- MSE:776
iniciando: BERTIOGA - SÃO LOURENÇO (RUA 2)
Treinando modelo: BERTIOGA-SÃO LOURENÇO (RUA 2)
Modelo Treinado: BERTIOGA-SÃO LOURENÇO (RUA 2)- MSE:1078
iniciando: BERTIOGA - ENSEADA - INDAIÁ
Treinando modelo: BERTIOGA-ENSEADA - INDAIÁ
Modelo Treinado: BERTIOGA-ENSEADA - INDAIÁ- MSE:2442
iniciando: BERTIOGA - ENSEADA - VISTA LINDA
Treinando modelo: BERTIOGA-ENSEADA - VISTA LINDA
Modelo Treinado: BERTIOGA-ENSEADA - VISTA LINDA-

Modelo Treinado: ITANHAÉM-BALNEÁRIO GAIVOTA- MSE:11136
iniciando: MONGAGUÁ - ITAPOÃ - VILA SÃO PAULO
Treinando modelo: MONGAGUÁ-ITAPOÃ - VILA SÃO PAULO
Modelo Treinado: MONGAGUÁ-ITAPOÃ - VILA SÃO PAULO- MSE:14862
iniciando: MONGAGUÁ - CENTRAL
Treinando modelo: MONGAGUÁ-CENTRAL
Modelo Treinado: MONGAGUÁ-CENTRAL- MSE:16315
iniciando: MONGAGUÁ - VERA CRUZ
Treinando modelo: MONGAGUÁ-VERA CRUZ
Modelo Treinado: MONGAGUÁ-VERA CRUZ- MSE:17635
iniciando: MONGAGUÁ - SANTA EUGÊNIA
Treinando modelo: MONGAGUÁ-SANTA EUGÊNIA
Modelo Treinado: MONGAGUÁ-SANTA EUGÊNIA- MSE:16945
iniciando: MONGAGUÁ - ITAÓCA
Treinando modelo: MONGAGUÁ-ITAÓCA
Modelo Treinado: MONGAGUÁ-ITAÓCA- MSE:15098
iniciando: MONGAGUÁ - AGENOR DE CAMPOS
Treinando modelo: MONGAGUÁ-AGENOR DE CAMPOS
Modelo Treinado: MONGAGUÁ-AGENOR DE CAMPOS- MSE:11826
iniciando: MONGAGUÁ - FLÓRIDA MIRIM
Treinando modelo: MONGAGUÁ-FLÓRIDA MIRIM
Modelo Treinado: MONGAGUÁ-FLÓRIDA MIRIM- MSE:10767
iniciando: PERUÍBE - PERUÍBE (R. ICARAÍBA)
Treinando modelo: 

Modelo Treinado: SÃO VICENTE-PRAIA DA DIVISA- MSE:20643
iniciando: SÃO VICENTE - ITARARÉ (POSTO 2)
Treinando modelo: SÃO VICENTE-ITARARÉ (POSTO 2)
Modelo Treinado: SÃO VICENTE-ITARARÉ (POSTO 2)- MSE:12967
iniciando: SÃO VICENTE - PRAIA DA ILHA PORCHAT
Treinando modelo: SÃO VICENTE-PRAIA DA ILHA PORCHAT
Modelo Treinado: SÃO VICENTE-PRAIA DA ILHA PORCHAT- MSE:8358
iniciando: SÃO VICENTE - MILIONÁRIOS
Treinando modelo: SÃO VICENTE-MILIONÁRIOS
Modelo Treinado: SÃO VICENTE-MILIONÁRIOS- MSE:290299
iniciando: SÃO VICENTE - GONZAGUINHA
Treinando modelo: SÃO VICENTE-GONZAGUINHA
Modelo Treinado: SÃO VICENTE-GONZAGUINHA- MSE:182279
iniciando: SÃO VICENTE - PRAINHA (AV. SANTINO BRITO)
Treinando modelo: SÃO VICENTE-PRAINHA (AV. SANTINO BRITO)
Modelo Treinado: SÃO VICENTE-PRAINHA (AV. SANTINO BRITO)- MSE:286449
iniciando: UBATUBA - PICINGUABA
Treinando modelo: UBATUBA-PICINGUABA
Modelo Treinado: UBATUBA-PICINGUABA- MSE:7436
iniciando: UBATUBA - FÉLIX
Treinando modelo: UBATUBA-FÉLIX
Modelo Treinado: 